# Intrusion Detection System using Deep Neural Networks (DNN)
This notebook trains and evaluates a DNN-based classifier for intrusion detection.


In [ ]:
# !pip install -r requirements.txt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras import layers, models
print(tf.__version__)


## 1) Load Data
Replace the paths below with your actual dataset files (e.g., `dataset/train.csv`).

In [ ]:
DATA_PATH = 'dataset/train.csv'  # Update to your file
df = pd.read_csv(DATA_PATH)
df.head()


## 2) Preprocess
Adjust feature/label column names as needed.

In [ ]:
target_col = 'label'
X = df.drop(columns=[target_col])
y = df[target_col]
if y.dtype == 'object':
    le = LabelEncoder(); y = le.fit_transform(y)
scaler = StandardScaler(); X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)
X_train.shape, X_test.shape


## 3) Build Model

In [ ]:
input_dim = X_train.shape[1]
model = models.Sequential([
    layers.Input(shape=(input_dim,)),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


## 4) Train

In [ ]:
history = model.fit(X_train, y_train, validation_split=0.2, epochs=20, batch_size=256, verbose=1)


## 5) Evaluate

In [ ]:
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {acc:.4f}")
y_pred = (model.predict(X_test) > 0.5).astype(int)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


## 6) Save Plots & Reports

In [ ]:
import json, matplotlib.pyplot as plt
plt.figure(); plt.plot(history.history['accuracy'], label='train_acc'); plt.plot(history.history['val_accuracy'], label='val_acc')
plt.title('Accuracy over Epochs'); plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.legend(); plt.savefig('results/accuracy.png', dpi=150); plt.close()
with open('results/metrics.json','w') as f: json.dump({'test_accuracy': float(acc)}, f, indent=2)
